### Sort and Convert to FASTQ

Sorts a BAM file and converts to FASTQ.
The input file is first checked for paired reads count. If its non zero its treated as a paired-end file.
Based on the user flag to split into R1 and R2 the paired file is separated into two fastq file. Else a single fastq file is created.

In [ ]:

import os
import subprocess
import sys

import pysam
from hops import hdfs
from pyspark import SparkContext

import utils
import time



#### load arguments

In [ ]:
#args=utils.load_arguments([0,'hdfs:///Projects/TCGA_viruses/Jupyter/pipeline/settings_DJ.yml'])
args=utils.load_arguments(sys.argv)
if args is not None:    
    args=args[utils.KEY_SORTCONVERT]
else :
    sys.exit(utils.NO_CONFIG_ERR)
    
sc = SparkContext.getOrCreate()

SPACE=utils.SPACE

inputRoot=args['INPUT_ROOT']
OUTPUT_FASTQ=args['OUTPUT_FASTQ']
THREADS=str(args['THREADS'])
# split separate R1 and R2 if paired file
is_split_R1R2=args['SPLIT_PAIRS']


#### Helper functions

In [ ]:
"""
Count paired  in BAM reads countfile using pysam.
Returns True if non zero.
"""
def count_paired_reads(x):
     
    o=pysam.view(x, '-c','-f','1',catch_stdout=True)
    if int(o)!=0: # if non zero paired reads count
        return True
    
    return False


"""
Counts paired reads count in BAM using samtools.
Returns True if non zero.
"""
def isPaired(file):

    cmd1=utils.SAMTOOLS+' view -c -f 1 '+file 
    print(cmd1)
    p1=subprocess.Popen(cmd1.split(),stdout=subprocess.PIPE)
    cmd2='head -n 1'
    p2=subprocess.run(cmd2.split(),stdin=p1.stdout,stdout=subprocess.PIPE)
    if p1.stdout  :
        return True
    else :
        return False


"""
Sort given BAM file using pysam
"""
def sort(file):        
    sort_file = utils.SORTED_PREFIX+file
    pysam.sort('-@',THREADS,'-m','2G','-n',file,'-o',sort_file,catch_stdout=False)
    return sort_file


"""
Map function to run on single BAM file.
First the file is sorted and then the sorted file is converted to 
single or paired FASTQ files depending on user argument 
and if file is paired or not.
Output is copied back to hdfs.
If output file name is already present in destination the process is skipped 
to avoid processing of same file incase of resubmit of failed run.


P.S/TODO: Commented code is to run via samtools and stays due to hadoop18 problem 
of running programs via subproces.
Once hadoop18 is fixed samtools can be uncommented and pysam code then can be subsequently deleted
"""
def sort_convert_fastq(file_path):
    file=os.path.split(file_path)[1]
    filename=os.path.splitext(file)[0]      
    output=filename+'.fq.gz'
    if not hdfs.exists(os.path.join(OUTPUT_FASTQ,output)): # check if output already exists
               
    
        hdfs.copy_to_local(file_path, overwrite=True)
         
        out_file_r1=''
        out_file_r2=''
        out_file=''


         # check if bam file has paired reads
        isPairedFile=count_paired_reads(file)
        print('is paired ? ', isPairedFile)

        # sort 
        print("INFO: Run sort ", file)    
        sort_file=sort(file)
        os.remove(file)
        os.rename(sort_file,sort_file.split(utils.SORTED_PREFIX)[1])

        # if the bam file is paired and user wants to split into separate R1 and R2 fastq
        if (isPairedFile and is_split_R1R2):
            out_file_r1=filename+'_R1.fq.gz'
            out_file_r2=filename+'_R2.fq.gz'
            #params={'fastq':file,'-1':out_file_r1,'-2':out_file_r2, '-0':'/dev/null', '-s':'/dev/null', '-c':1 ,'-@':THREADS}    
            pysam.fastq(file,'-1',out_file_r1,'-2',out_file_r2, '-0','/dev/null', '-s','/dev/null', '-c','1','-@',THREADS,'-N',catch_stdout=False)


        else:   # else if the file is single read or the user wants to split into a single fastq      
            out_file=filename+'.fq.gz'
            #params={'fastq': file,'-0': out_file, '-s':'/dev/null', '-c':1, '-@': THREADS}
            if isPairedFile: # samtools fastq has two different argument values for output if the file is paired or not
                OUT_ARG='-o'
            else :
                OUT_ARG='-0'
            pysam.fastq(file, OUT_ARG, out_file,  '-c','1','-@',THREADS,'-N',catch_stdout=False)


    # convert to fastq command  using samtools
    #     cmdConvert=utils.build_command(utils.SAMTOOLS,params)  
    #     print('INFO: Run sort and convert', cmdConvert)
    #     p1 = subprocess.run(cmdConvert.split(), stdout=subprocess.PIPE) # run convert from pipe from sort command

        os.remove(file)
        if os.path.exists(out_file): # either a single fastq exists or separate R1 R2 fastq files
            hdfs.copy_to_hdfs(out_file,OUTPUT_FASTQ,overwrite=True)
            os.remove(out_file)
            return True
        if os.path.exists(out_file_r1):
            hdfs.copy_to_hdfs(out_file_r1,OUTPUT_FASTQ,overwrite=True)
            hdfs.copy_to_hdfs(out_file_r2,OUTPUT_FASTQ,overwrite=True)
            os.remove(out_file_r1)
            os.remove(out_file_r2)
            return True


        return False
    
    else:
        print('skipping existing file: ', filename)            
        return None

        
     
    
    
    
    



#### Load input files hdfs paths

In [ ]:
inputFiles=utils.load_file_names(inputRoot)


#### Run in parallel

In [ ]:


finalList = sc.parallelize(inputFiles).map(sort_convert_fastq).collect()
